Importing Required Libraries

In [25]:
import pandas as pd 
import numpy as np 
import os
from sqlalchemy import create_engine
import pymysql
from sqlalchemy import text

In [7]:
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')

In [11]:
host = 'localhost'
database = input('Enter database name : ')

Enter database name :  cust


Importing data

In [9]:
clicks = pd.read_csv('clicks_data.csv')
customers = pd.read_csv('customers_data.csv')
discount = pd.read_csv('discount_data.csv')
product = pd.read_csv('product_data.csv')

In [59]:
clicks.head(10)

,click_id,customer_id,product_id,campaign_id,click_date,device_type,order_id,session_id
0,31cfda10-f2cb-418c-853a-5956f0581050,699.0,2,NaN,2023-07-23 00:00:00,Mobile,NaN,9415d085-0d8a-4ae8-99de-2e74e63296da
1,9347e6e5-169d-48b0-b27b-e30ea6988b6a,517.0,2,NaN,2022-08-16 00:00:00,Desktop,NaN,9415d085-0d8a-4ae8-99de-2e74e63296da
2,80dcac65-db29-442a-89be-d45eda9e1217,988.0,4,NaN,2022-07-09 00:00:00,Desktop,NaN,7e21ad09-8d50-4216-95b4-adb88967fdba
3,8fcb7398-e403-410a-91fe-3bc702911c9d,972.0,12,NaN,2022-09-06 00:00:00,Tablet,NaN,7e21ad09-8d50-4216-95b4-adb88967fdba
4,7245e7fc-3ff1-4c15-a271-b762fb9bf41c,390.0,5,1562d187-f4e7-4ea4-92ad-4ef57621abcd,2021-12-03 00:00:00,Mobile,NaN,4014c257-9f49-4a33-8853-5d832563c55e
5,09399661-dc6c-4695-b8f6-688384f829bd,1017.0,11,NaN,2021-07-02 00:00:00,Desktop,NaN,07d84195-44c1-414f-9340-a924f7abc539
6,6cb6e598-d699-453c-93c1-dedfa4444374,545.0,10,NaN,2021-12-05 00:00:00,Desktop,NaN,07d84195-44c1-414f-9340-a924f7abc539
7,da67908a-7f9b-4f8b-b8cd-1ad6af5eebd1,257.0,3,NaN,2021-10-03 00:00:00,Mobile,NaN,07d84195-44c1-414f-9340-a924f7abc539
8,f4fb5b0b-f06e-42c6-ba26-cd457f252e44,698.0,7,9f45e31e-826f-48d1-8e93-f4edb9c17b7d,2023-09-18 00:00:00,Mobile,NaN,07d84195-44c1-414f-9340-a924f7abc539
9,e14f3660-4c9a-40e2-bfdb-af363a89f090,890.0,12,7031c599-1483-4186-826e-dd7d4d8a1e96,2022-12-08 00:00:00,Mobile,NaN,52545f03-d425-4d43-9f97-3c915e3723af


In [61]:
customers.head(10)

,customer_id,name,street_address,state,city,zipcode,phone_number
0,1,Mrs. Kristina Shaw,213 Gloria Road,TX,San Antonio,26126,(595) 905-3764
1,2,Amy Gallegos,292 Benson Manors,VA,Richmond,89487,(458) 732-3164
2,3,Christina Hubbard,0557 White Ways Suite 730,PA,Reading,86704,(856) 036-8738
3,4,Elizabeth Edwards,5675 Nicole Gateway,TX,Austin,48129,(198) 825-0273
4,5,Frank Mays,4387 Andrea Mall,AZ,Chandler,86152,(918) 111-2066
5,6,Alexandra Daniel,259 David Haven Suite 510,NY,Albany,82662,(306) 766-5682
6,7,Joshua Ross,2038 Angelica Ports,FL,Tampa,59292,(765) 876-2826
7,8,Kimberly Quinn,36616 Richard Ridges Apt. 608,MI,Warren,82175,(729) 085-2073
8,9,Katelyn Pope,63596 Cody Cape Suite 059,VA,Chesapeake,34953,(741) 583-3166
9,10,Monica Olson,11437 Hoffman Shoal Suite 484,GA,Savannah,51986,(638) 578-8754


In [56]:
discount['offer_percent'] = discount['discount_description'].str.extract(r'(\d+)%').astype(int)
discount.head(10)

,discount_id,discount_description,discount_type,start_date,end_date,offer_percent
0,55a8e33c-cd15-4bbe-a0f6-ea5c9da1b201,Seasonal Sale - Enjoy 10% off on selected items!,0.10,2021-10-05,2021-11-19,10
1,df25f0cc-5e25-4260-8e5a-60029850596a,Flash Sale - Enjoy 50% off on selected items!,0.50,2022-06-01,2023-10-24,50
2,f289e5cd-fa7e-44ab-8fd3-9a36b1bf3c59,Holiday Sale - Enjoy 25% off on selected items!,0.25,2023-09-12,2023-10-15,25
3,d0b176f9-9711-404d-836d-c5e2eee9d416,Flash Sale - Enjoy 10% off on selected items!,0.10,2023-02-16,2023-03-24,10
4,858e4b74-ece4-45c4-90d9-7ebbfe779ad9,Seasonal Sale - Enjoy 15% off on selected items!,0.15,2021-12-08,2021-12-14,15
5,4b569a72-9362-461c-8ed2-aab870d76ec6,End of Season Clearance - Enjoy 15% off on sel...,0.15,2022-06-20,2023-11-01,15
6,683a8480-9e38-428d-93e2-babbfac5d54c,Anniversary Sale - Enjoy 15% off on selected i...,0.15,2021-10-23,2022-04-10,15
7,c7585c13-5c81-4381-bef9-f5eb2706a98f,Seasonal Sale - Enjoy 10% off on selected items!,0.10,2021-12-19,2023-05-03,10
8,b2d9921c-9b4f-4de2-93c9-f92ff7a9fd76,Holiday Sale - Enjoy 20% off on selected items!,0.20,2022-03-09,2023-08-20,20
9,17cca8c1-610a-437e-8abf-b300d5c0d2e1,Holiday Sale - Enjoy 25% off on selected items!,0.25,2023-12-15,2024-01-03,25


In [63]:
product.head(10)

,product_id,product_name,purchase_price,sales_price,active_status
0,2,Stainless Steel Water Bottle,21.44,49.81,True
1,3,Ergonomic Wireless Mouse,17.85,44.32,True
2,4,Solar-Powered Garden Light,31.22,39.31,True
3,5,Noise-Cancelling Headphones,178.97,432.16,False
4,6,Automatic Espresso Machine,299.02,565.50,True
5,7,Smartphone Tripod Stand,26.16,34.51,True
6,8,Reusable Silicone Baking Mats,14.67,26.84,True
7,9,Foldable Solar Charger,92.22,169.38,True
8,10,Electric Toothbrush with UV Sanitizer,47.35,87.20,True
9,11,Robotic Vacuum Cleaner,185.33,462.17,False


Creating database connection

In [13]:
engine = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{host}/{database}')

Replace actual dataframes and table names

In [17]:
datasets = [(clicks, 'clicks_data'), (customers, 'customers_data'), (discount, 'discount_data'), (product, 'product_data')]

Insert data into SQL tables

In [21]:
for df, table_name in datasets:
    try:
        print(f"✅ Ready to insert into '{table_name}' with shape {df.shape}")
        df.to_sql(name=table_name, con=engine, index=False, if_exists='replace')
        print(f"📁 Inserted into table '{table_name}' successfully.")
    except Exception as e:
        print(f"❌ Failed to insert into table '{table_name}': {e}")

✅ Ready to insert into 'clicks_data' with shape (126000, 8)
📁 Inserted into table 'clicks_data' successfully.
✅ Ready to insert into 'customers_data' with shape (1030, 7)
📁 Inserted into table 'customers_data' successfully.
✅ Ready to insert into 'discount_data' with shape (100, 5)
📁 Inserted into table 'discount_data' successfully.
✅ Ready to insert into 'product_data' with shape (11, 5)
📁 Inserted into table 'product_data' successfully.


In [29]:
select_query = "SELECT * FROM product_data;"

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    for row in result:
        print(row)

(2, 'Stainless Steel Water Bottle', 21.44, 49.81, 1)
(3, 'Ergonomic Wireless Mouse', 17.85, 44.32, 1)
(4, 'Solar-Powered Garden Light', 31.22, 39.31, 1)
(5, 'Noise-Cancelling Headphones', 178.97, 432.16, 0)
(6, 'Automatic Espresso Machine', 299.02, 565.5, 1)
(7, 'Smartphone Tripod Stand', 26.16, 34.51, 1)
(8, 'Reusable Silicone Baking Mats', 14.67, 26.84, 1)
(9, 'Foldable Solar Charger', 92.22, 169.38, 1)
(10, 'Electric Toothbrush with UV Sanitizer', 47.35, 87.2, 1)
(11, 'Robotic Vacuum Cleaner', 185.33, 462.17, 0)
(12, 'Smart Light Bulb Kit', 67.33, 94.22, 1)


In [31]:
select_query = "Select * from clicks_data where device_type = 'Mobile';"

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    for row in result:
        print(row)

('31cfda10-f2cb-418c-853a-5956f0581050', 699.0, 2, None, '2023-07-23 00:00:00', 'Mobile', None, '9415d085-0d8a-4ae8-99de-2e74e63296da')
('7245e7fc-3ff1-4c15-a271-b762fb9bf41c', 390.0, 5, '1562d187-f4e7-4ea4-92ad-4ef57621abcd', '2021-12-03 00:00:00', 'Mobile', None, '4014c257-9f49-4a33-8853-5d832563c55e')
('da67908a-7f9b-4f8b-b8cd-1ad6af5eebd1', 257.0, 3, None, '2021-10-03 00:00:00', 'Mobile', None, '07d84195-44c1-414f-9340-a924f7abc539')
('f4fb5b0b-f06e-42c6-ba26-cd457f252e44', 698.0, 7, '9f45e31e-826f-48d1-8e93-f4edb9c17b7d', '2023-09-18 00:00:00', 'Mobile', None, '07d84195-44c1-414f-9340-a924f7abc539')
('e14f3660-4c9a-40e2-bfdb-af363a89f090', 890.0, 12, '7031c599-1483-4186-826e-dd7d4d8a1e96', '2022-12-08 00:00:00', 'Mobile', None, '52545f03-d425-4d43-9f97-3c915e3723af')
('e6d7c6e3-5816-4aaa-b47a-cb449b38cbee', None, 10, None, '2021-09-04 00:00:00', 'Mobile', None, '52545f03-d425-4d43-9f97-3c915e3723af')
('41687628-651f-4639-b77c-6bd18fb1156b', 791.0, 6, None, '2023-01-21 00:00:00', '

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

